# 02c - Vertex AI > Pipelines - AutoML with clients (code) In automated pipeline

Use[ Kubeflow](https://www.kubeflow.org/) Pipelines running on [Vertex AI Pipelines](https://cloud.google.com/vertex-ai/docs/pipelines/introduction) to orchestrate the process of training a custom model with AutoML Tabular and deploy it to a Vertex AI Endpoint for serving (online and batch) predictions and explanations.  This demonstrates how to automate the processes of (02a) or (02b) with pipeline orchestration.

**Prerequisites:**

-  01 -  BigQuery - Table Data Source

**Overview:**

-  Use Kubeflow Python SDK to build a pipeline
   -  Create pipeline using Google Cloud Pipeline Components (from google_cloud_pipeline_components import aiplatform as gcc_aip)
      -  Use cc_aip.TabularDatasetCreateOp to register dataset from BigQuery table
      -  Train AutoML tabular model with gcc_aip.AutoMLTabularTrainingJobRunOp
      -  Deploy model to endpoint using gcc_aip.ModelDeployOp
   -  Compile the pipeline
      -  kfp.v2.compiler.Compiler().compile
   -  Move the pipeline code to GCS Bucket
   -  Run the pipeline with google.cloud.aiplatform.PipelineJob
-  Online Predictions using Vertex AI Endpoint
-  Online Explanations using Vertex AI Endpoint
-  Batch Prediction Job for predictions and explanation with source and destination tables in BigQuery

**Resources:**

-  [Vertex AI Pipelines](https://cloud.google.com/vertex-ai/docs/pipelines/build-pipeline#google-cloud-components) see aiplatform.PipelineJob
-  [Python Client for Vertex AI](https://googleapis.dev/python/aiplatform/latest/aiplatform.html)
-  [Kubeflow Pipelines Components for Google Cloud](https://github.com/kubeflow/pipelines/tree/master/components/google-cloud)

**Related Training:**

-  Codelab: [Vertex AI Pipelines Introduction](https://codelabs.developers.google.com/vertex-mlmd-pipelines#0)
-  todo

---
## Conceptual Architecture

<img src="architectures/statmike-mlops-A2.png">

---
## Setup

inputs:

In [1]:
REGION = 'us-central1'
PROJECT_ID='statmike-mlops'
DATANAME = 'digits'
NOTEBOOK = '02c'

# Resources
DEPLOY_COMPUTE = 'n1-standard-4'

packages:

In [2]:
from google.cloud import aiplatform
from datetime import datetime
import kfp
import kfp.v2.dsl as dsl
from google_cloud_pipeline_components import aiplatform as gcc_aip

from google.cloud import bigquery
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value

import numpy as np

clients:

In [3]:
aiplatform.init(project=PROJECT_ID, location=REGION)
bigquery = bigquery.Client()

parameters:

In [4]:
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
BUCKET = PROJECT_ID
URI = f"gs://{BUCKET}/{DATANAME}/models/{NOTEBOOK}"
DIR = f"temp/{NOTEBOOK}"

Give service account roles/storage.objectAdmin permissions: Console > IAM > Select account `<projectnumber>-compute@developer.gserviceaccount.com` > edit - give it the role

In [5]:
SERVICE_ACCOUNT = !gcloud config list --format='value(core.account)' 
SERVICE_ACCOUNT = SERVICE_ACCOUNT[0]
SERVICE_ACCOUNT

'691911073727-compute@developer.gserviceaccount.com'

environment:

In [6]:
!rm -rf {DIR}
!mkdir -p {DIR}

---
## Pipeline (KFP) Definition
- Flow
    - Create Vertex AI Dataset from link to BigQuery table
    - Create Vertex AI AutoML Tabular Training Job
    - Create Endpoint and Depoy trained model

In [7]:
@kfp.dsl.pipeline(name = f'kfp-{NOTEBOOK}-{DATANAME}-{TIMESTAMP}', pipeline_root = URI+'/'+str(TIMESTAMP)+'/kfp/')
def pipeline(
    project: str = PROJECT_ID,
    display_name: str = f'{NOTEBOOK}_{DATANAME}_{TIMESTAMP}',
    deploy_machine: str = DEPLOY_COMPUTE,
    bq_source: str = f'bq://{PROJECT_ID}.{DATANAME}.{DATANAME}_prepped',
    label: str = NOTEBOOK 
):
    
    # dataset
    dataset = gcc_aip.TabularDatasetCreateOp(
        project = project,
        display_name = display_name,
        bq_source = bq_source,
        labels = {'notebook':f'{label}'}
    )
    
    # The feature columns are named `p0` through `p63`. Using this naming, the column_specs dictionary of datatypes can be defined:
    column_specs = {}
    for i in range(64):
        column_specs['p'+str(i)] = 'auto'
    
    # training
    model = gcc_aip.AutoMLTabularTrainingJobRunOp(
        project = project,
        display_name = display_name,
        optimization_prediction_type = "classification",
        budget_milli_node_hours = 1000,
        disable_early_stopping=False,
        column_specs = column_specs,
        dataset = dataset.outputs['dataset'],
        target_column = "target",
        #predefined_split_column_name = 'splits',
        training_fraction_split = 0.8,
        validation_fraction_split = 0.1,
        test_fraction_split = 0.1,
        labels = {'notebook':f'{label}'}
    )
    
    # Endpoint Deployment
    endpoint = gcc_aip.ModelDeployOp(
        model = model.outputs["model"],
        project = project,
        #display_name = display_name,
        machine_type = deploy_machine,
        #labels = {'notebook':f'{label}'}
    )

---
## Compile Pipeline

In [8]:
kfp.v2.compiler.Compiler().compile(
    pipeline_func = pipeline,
    package_path = f"{DIR}/{NOTEBOOK}.json"
)

Move compiled pipeline files to GCS Bucket

In [9]:
!gsutil cp {DIR}/{NOTEBOOK}.json {URI}/{TIMESTAMP}/kfp/

Copying file://temp/02c/02c.json [Content-Type=application/json]...
/ [1 files][ 11.8 KiB/ 11.8 KiB]                                                
Operation completed over 1 objects/11.8 KiB.                                     


---
## Create Vertex AI Pipeline Job

In [10]:
pipeline = aiplatform.PipelineJob(
    display_name = f'{NOTEBOOK}_{DATANAME}_{TIMESTAMP}',
    template_path = f"{URI}/{TIMESTAMP}/kfp/{NOTEBOOK}.json",
    pipeline_root = f"{URI}/{TIMESTAMP}/kfp/",
    labels = {'notebook':f'{NOTEBOOK}'}
)

In [37]:
response = pipeline.run(service_account = SERVICE_ACCOUNT)

INFO:google.cloud.aiplatform.pipeline_jobs:Creating PipelineJob
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob created. Resource name: projects/691911073727/locations/us-central1/pipelineJobs/kfp-02c-digits-20210915141811-20210915142043
INFO:google.cloud.aiplatform.pipeline_jobs:To use this PipelineJob in another session:
INFO:google.cloud.aiplatform.pipeline_jobs:pipeline_job = aiplatform.PipelineJob.get('projects/691911073727/locations/us-central1/pipelineJobs/kfp-02c-digits-20210915141811-20210915142043')
INFO:google.cloud.aiplatform.pipeline_jobs:View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/kfp-02c-digits-20210915141811-20210915142043?project=691911073727
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob projects/691911073727/locations/us-central1/pipelineJobs/kfp-02c-digits-20210915141811-20210915142043 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob projec

In [38]:
aiplatform.get_pipeline_df(pipeline = f'kfp-{NOTEBOOK}-{DATANAME}-{TIMESTAMP}')

,pipeline_name,run_name,param.input:label,param.input:project,param.input:bq_source,param.input:display_name,param.input:deploy_machine
0,kfp-02c-digits-20210915141811,kfp-02c-digits-20210915141811-20210915142043,02c,statmike-mlops,bq://statmike-mlops.digits.digits_prepped,02c_digits_20210915141811,n1-standard-4


---
## Prediction

### Prepare a record for prediction: instance and parameters lists

In [6]:
pred = bigquery.query(query = f"SELECT * FROM {DATANAME}.{DATANAME} LIMIT 10").to_dataframe()

In [7]:
pred.head(4)

,p0,p1,p2,p3,p4,p5,p6,p7,p8,p9,...,p56,p57,p58,p59,p60,p61,p62,p63,target,target_OE
0,0.0,5.0,16.0,15.0,5.0,0.0,0.0,0.0,0.0,2.0,...,0.0,6.0,16.0,16.0,16.0,16.0,7.0,0.0,2,Even
1,0.0,5.0,16.0,12.0,1.0,0.0,0.0,0.0,0.0,5.0,...,0.0,8.0,16.0,16.0,16.0,16.0,4.0,0.0,2,Even
2,0.0,5.0,15.0,16.0,6.0,0.0,0.0,0.0,0.0,11.0,...,0.0,6.0,16.0,16.0,16.0,13.0,3.0,0.0,2,Even
3,0.0,4.0,15.0,15.0,8.0,0.0,0.0,0.0,0.0,8.0,...,0.0,7.0,14.0,11.0,0.0,0.0,0.0,0.0,2,Even


In [41]:
newob = pred.loc[:0,'p0':'p63'].to_dict(orient='records')[0]
#newob

In [44]:
instances = [json_format.ParseDict(newob, Value())]
parameters = json_format.ParseDict({}, Value())

### Prediction: Online

In [56]:
endpoint = aiplatform.Endpoint.list(filter=f'display_name={NOTEBOOK}_{DATANAME}_{TIMESTAMP}_endpoint')[0]
endpoint.display_name

'02c_digits_20210915141811_endpoint'

In [57]:
prediction = endpoint.predict(instances=instances, parameters=parameters)
prediction

Prediction(predictions=[{'scores': [1.632638191306768e-10, 1.571539842704084e-10, 6.781004315561067e-16, 8.908038830234347e-14, 1.262391258194384e-08, 3.106381102189635e-16, 1.514301739558022e-10, 3.003936617680669e-13, 1.0, 4.828001605276799e-13], 'classes': ['1', '7', '6', '9', '8', '4', '3', '0', '2', '5']}], deployed_model_id='1446297595176550400', explanations=None)

In [60]:
prediction.predictions[0]['classes'][np.argmax(prediction.predictions[0]['scores'])]

'2'

## Explanations
Interpretation Guide
- https://cloud.google.com/vertex-ai/docs/predictions/interpreting-results-automl#tabular

In [61]:
explanation = endpoint.explain(instances=instances, parameters=parameters)

In [62]:
explanation.predictions

[{'scores': [1.632638191306768e-10,
   1.571539842704084e-10,
   6.781004315561067e-16,
   8.908038830234347e-14,
   1.262391258194384e-08,
   3.106381102189635e-16,
   1.514301739558022e-10,
   3.003936617680669e-13,
   1.0,
   4.828001605276799e-13],
  'classes': ['1', '7', '6', '9', '8', '4', '3', '0', '2', '5']}]

In [63]:
print("attribution:")
print("baseline output",explanation.explanations[0].attributions[0].baseline_output_value)
print("instance output",explanation.explanations[0].attributions[0].instance_output_value)
print("output_index",explanation.explanations[0].attributions[0].output_index)
print("output display value",explanation.explanations[0].attributions[0].output_display_name)
print("approximation error",explanation.explanations[0].attributions[0].approximation_error)

attribution:
baseline output 0.0032008790876716375
instance output 1.0
output_index [8]
output display value 2
approximation error 0.026432039432736847


In [64]:
explanation.explanations[0].attributions[0]

baseline_output_value: 0.0032008790876716375
instance_output_value: 1.0
feature_attributions {
  struct_value {
    fields {
      key: "p0"
      value {
        number_value: 0.0
      }
    }
    fields {
      key: "p1"
      value {
        number_value: 0.007557083935742932
      }
    }
    fields {
      key: "p10"
      value {
        number_value: 0.0
      }
    }
    fields {
      key: "p11"
      value {
        number_value: 0.0001349432526954583
      }
    }
    fields {
      key: "p12"
      value {
        number_value: 0.008827902027405798
      }
    }
    fields {
      key: "p13"
      value {
        number_value: 0.08300510830512005
      }
    }
    fields {
      key: "p14"
      value {
        number_value: 0.0
      }
    }
    fields {
      key: "p15"
      value {
        number_value: 0.0
      }
    }
    fields {
      key: "p16"
      value {
        number_value: 0.0
      }
    }
    fields {
      key: "p17"
      value {
        number_value: 

### Batch Predictions: BigQuery Source to BigQuery Destination, with Explanations

In [67]:
batch = aiplatform.BatchPredictionJob.create(
    job_display_name = f'{NOTEBOOK}_{DATANAME}_{TIMESTAMP}',
    model_name = endpoint.list_models()[0].model,
    instances_format = "bigquery",
    predictions_format = "bigquery",
    bigquery_source = f'bq://{PROJECT_ID}.{DATANAME}.{DATANAME}',
    bigquery_destination_prefix = f"{PROJECT_ID}",
    generate_explanation=True,
    labels = {'notebook':f'{NOTEBOOK}'}
)

INFO:google.cloud.aiplatform.jobs:Creating BatchPredictionJob
INFO:google.cloud.aiplatform.jobs:BatchPredictionJob created. Resource name: projects/691911073727/locations/us-central1/batchPredictionJobs/7411572587349671936
INFO:google.cloud.aiplatform.jobs:To use this BatchPredictionJob in another session:
INFO:google.cloud.aiplatform.jobs:bpj = aiplatform.BatchPredictionJob('projects/691911073727/locations/us-central1/batchPredictionJobs/7411572587349671936')
INFO:google.cloud.aiplatform.jobs:View Batch Prediction Job:
https://console.cloud.google.com/ai/platform/locations/us-central1/batch-predictions/7411572587349671936?project=691911073727
INFO:google.cloud.aiplatform.jobs:BatchPredictionJob projects/691911073727/locations/us-central1/batchPredictionJobs/7411572587349671936 current state:
JobState.JOB_STATE_RUNNING
INFO:google.cloud.aiplatform.jobs:BatchPredictionJob projects/691911073727/locations/us-central1/batchPredictionJobs/7411572587349671936 current state:
JobState.JOB_STAT